In [1]:
# Basic Imports
import pickle
import os
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import folium
from tqdm import tqdm

# Geo
from shapely.geometry import box, Polygon
import geopandas as gpd

# Rasterio
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
import rioxarray


In [2]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
city_3d_model_path = config['data']['city_3d_model']
ortho_path = config['data']['orthophotos']
rooftop_tifs_path = config['data']['rooftop_tifs']

### Import 3D City model data

In [3]:
with open(city_3d_model_path + 'processed/processed_roofs.pkl', 'rb') as f:
    gdf = pickle.load(f)

# Subset to only flat roofs
gdf = gdf[gdf.roofType.isin([1000])]
# Subset to only inner city
bbox = box(*[11.557617,48.123591,11.599846,48.147649])
gdf = gdf[gdf.geometry_4326.intersects(bbox)]
# Use only buildings with height > 3 meter to get rid of small objects like bus stops
#gdf = gdf[gdf.measuredHeight > 3]
# Get rid of bridges and Überdachungen
gdf = gdf[gdf.function.isin(['51009_1610', '53001_1800']) == False]

# Susbet to relevant columns
gdf = gdf[['id', 'geometry']]

# Sort by area of roof 
gdf['area'] = gdf.geometry.area
gdf.sort_values('area', ascending=False, inplace=True)


gdf.head(3)

,id,geometry,area
3070,DEBY_LOD2_4955860,"MULTIPOLYGON (((692064.360 5334089.410, 692064...",12072.216919
3329,DEBY_LOD2_4958619,"MULTIPOLYGON (((691399.952 5335853.259, 691397...",8572.973060
3898,DEBY_LOD2_5007168,"MULTIPOLYGON (((692435.339 5335549.354, 692428...",8416.904426


### Import Orthophoto index


In [4]:
with open(ortho_path + '/metalink_files/complete_index.pkl', 'rb') as f:
    ortho_index = pickle.load(f)
ortho_index = gpd.GeoDataFrame(ortho_index)
ortho_index.set_geometry('polygon_25832', inplace=True)

In [5]:
def get_relevant_tiles(bbox):
    return list(ortho_index[ortho_index.polygon_25832.intersects(bbox)].tile_name)

In [6]:
gdf['relevant_tiles'] = gdf.geometry.map(get_relevant_tiles)
gdf.relevant_tiles.map(len).value_counts(normalize = True)*100

1    96.709280
2     3.243371
4     0.023674
3     0.023674
Name: relevant_tiles, dtype: float64

### Crop and save rooftop pictures from relevant tiles

In [15]:
import rasterio
from rasterio.mask import mask
from PIL import Image
import numpy as np

def crop_and_save(row, directory):
    tiles = row['relevant_tiles'] # retrieve list of tiles
    rooftop_geom = row.geometry
    # convert to GeoJSON-like dict
    rooftop_id = row['id']

    for i, tile_name in enumerate(tiles):
        with rasterio.open(f"{ortho_path}/raw_tiles/{tile_name}") as src: # replace with your path
            out_image, out_transform = mask(src, [rooftop_geom], crop=True)
            out_meta = src.meta.copy()
            
            # save the resulting raster  
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })

            with rasterio.open(f"{directory}/tifs/{rooftop_id}_{i}.tif", "w", **out_meta) as dest:
                dest.write(out_image)
            
            # also save png
            if out_image.shape[0] == 1:
                # Grayscale image, convert the 3D array to 2D
                im_arr = np.squeeze(out_image)
            else:
                # Possibly multiband image, shift the bands axis to the last position
                im_arr = np.moveaxis(out_image, 0, -1)

            im = Image.fromarray(im_arr)
            im.save(f"{directory}/pngs/{rooftop_id}_{i}.png")


In [16]:
# Applying the function to each row of the geodataframe
for idx, row in tqdm(gdf.iterrows()):
    crop_and_save(row, rooftop_tifs_path)

4224it [00:50, 83.18it/s]


### Check correctness

In [9]:
all_rooftops = glob(rooftop_tifs_path+'/tifs/*.tif')
len(all_rooftops)

4366

In [12]:
def add_roof_picture(path):
    roof_picture = rioxarray.open_rasterio(path)
    roof_picture = roof_picture.rio.reproject('EPSG:4326')
    img = np.dstack((roof_picture.values[0], roof_picture.values[1], roof_picture.values[2]))

    image_bounds = box(*roof_picture.rio.bounds())
    # Plot roof on the map
    folium.GeoJson(image_bounds.__geo_interface__).add_to(m)
    min_x, min_y, max_x, max_y = roof_picture.rio.bounds()

    corner_coordinates = [[min_y, min_x], [max_y, max_x]]
    folium.raster_layers.ImageOverlay(img, bounds=corner_coordinates,opacity=0.9).add_to(m)

# Plot rooftops
m = folium.Map(location = [48.14715022477909, 11.572241757463829], zoom_start = 15)
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(m)
for file in all_rooftops[:1000]:
    add_roof_picture(file)
folium.LayerControl().add_to(m)
